# Experiment 001: Baseline with Arrhenius Kinetics + TTA

Implementing the best known approach from public kernels:
- Physics-informed features (1/T, ln(t), interaction)
- Chemical symmetry TTA for mixed solvents
- 7 bagged MLP models
- HuberLoss, BatchNorm, Dropout, Sigmoid output

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import sys
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.set_default_dtype(torch.double)

CUDA available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB


In [2]:
# Data paths for local environment
DATA_PATH = '/home/data'

INPUT_LABELS_FULL_SOLVENT = [
    "Residence Time",
    "Temperature",
    "SOLVENT A NAME",
    "SOLVENT B NAME",
    "SolventB%",
]

INPUT_LABELS_SINGLE_SOLVENT = [
    "Residence Time",
    "Temperature",
    "SOLVENT NAME",
]

INPUT_LABELS_NUMERIC = [
    "Residence Time",
    "Temperature",
]

TARGET_LABELS = [
    "Product 2",
    "Product 3",
    "SM",
]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    features = pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)
    return features

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Spange descriptors shape: {SPANGE_DF.shape}")
print(f"Solvents: {list(SPANGE_DF.index)}")

Spange descriptors shape: (26, 13)
Solvents: ['Cyclohexane', 'Ethyl Acetate', 'Acetic Acid', '2-Methyltetrahydrofuran [2-MeTHF]', '1,1,1,3,3,3-Hexafluoropropan-2-ol', 'IPA [Propan-2-ol]', 'Ethanol', 'Methanol', 'Ethylene Glycol [1,2-Ethanediol]', 'Acetonitrile', 'Water', 'Diethyl Ether [Ether]', 'MTBE [tert-Butylmethylether]', 'Dimethyl Carbonate', 'tert-Butanol [2-Methylpropan-2-ol]', 'DMA [N,N-Dimethylacetamide]', '2,2,2-Trifluoroethanol', 'Dihydrolevoglucosenone (Cyrene)', 'Decanol', 'Butanone [MEK]', 'Ethyl Lactate', 'Methyl Propionate', 'THF [Tetrahydrofuran]', 'Water.Acetonitrile', 'Acetonitrile.Acetic Acid', 'Water.2,2,2-Trifluoroethanol']


In [3]:
# Load and inspect data
X_single, Y_single = load_data('single_solvent')
X_full, Y_full = load_data('full')

print(f"Single solvent data: {X_single.shape}, {Y_single.shape}")
print(f"Full data: {X_full.shape}, {Y_full.shape}")
print(f"\nUnique solvents (single): {X_single['SOLVENT NAME'].nunique()}")
print(f"Unique solvent ramps (full): {X_full[['SOLVENT A NAME', 'SOLVENT B NAME']].drop_duplicates().shape[0]}")

Single solvent data: (656, 3), (656, 3)
Full data: (1227, 5), (1227, 3)

Unique solvents (single): 24
Unique solvent ramps (full): 13


In [4]:
# Define the model classes following the template structure

class SmilesFeaturizer(ABC):
    def featurize(self, X, flip=False): 
        raise NotImplementedError

class KineticMixingFeaturizer(SmilesFeaturizer):
    """Featurizer with Arrhenius kinetics features"""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # 2 numeric + 3 kinetic + 13 spange = 18 features
        self.feats_dim = self.featurizer.shape[1] + 2 + 3

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # --- ARRHENIUS KINETIC FEATURES ---
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (Arrhenius)
        log_time = np.log(time_m + 1e-6)  # Log time
        interaction = inv_temp * log_time  # Interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # --- CHEMICAL FEATURES ---
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # SYMMETRY FLIP: Swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)


class MLPInternal(nn.Module):
    """MLP with BatchNorm, Dropout, Sigmoid output"""
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid()  # Constrain output to [0, 1]
        )

    def forward(self, x):
        return self.net(x)


class SymmetricBaggedModel(nn.Module):
    """Bagged MLP with TTA for chemical symmetry"""
    def __init__(self, data='single', n_models=7, epochs=300, verbose=False):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.epochs = epochs
        self.verbose = verbose
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train):
        # 1. Standard Data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # 2. Augmented Data (Train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss()  # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(self.epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        # --- TEST TIME AUGMENTATION (TTA) ---
        if self.data_type == 'full':
            # Path A: Standard
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            # Path B: Flipped (Symmetry)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    # Average the model's prediction on A and B views
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5 
            
            avg_pred = pred_sum / self.n_models
            
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

print("Model classes defined successfully!")

Model classes defined successfully!


In [5]:
# Quick test to verify model works
print("Testing model on a small subset...")

# Test single solvent
X_test, Y_test = load_data('single_solvent')
test_model = SymmetricBaggedModel(data='single', n_models=1, epochs=5)
test_model.train_model(X_test.head(100), Y_test.head(100))
preds = test_model.predict(X_test.head(10))
print(f"Single solvent test predictions shape: {preds.shape}")
print(f"Predictions range: [{preds.min():.3f}, {preds.max():.3f}]")

# Test full data
X_test_full, Y_test_full = load_data('full')
test_model_full = SymmetricBaggedModel(data='full', n_models=1, epochs=5)
test_model_full.train_model(X_test_full.head(100), Y_test_full.head(100))
preds_full = test_model_full.predict(X_test_full.head(10))
print(f"\nFull data test predictions shape: {preds_full.shape}")
print(f"Predictions range: [{preds_full.min():.3f}, {preds_full.max():.3f}]")

Testing model on a small subset...


Single solvent test predictions shape: torch.Size([10, 3])
Predictions range: [0.399, 0.566]

Full data test predictions shape: torch.Size([10, 3])
Predictions range: [0.351, 0.840]


In [6]:
# Run full cross-validation for single solvent task
print("\n" + "="*60)
print("TASK 0: Single Solvent (Leave-One-Solvent-Out CV)")
print("="*60)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []
fold_mses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    # Use fewer models and epochs for faster iteration
    model = SymmetricBaggedModel(data='single', n_models=5, epochs=200)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    actuals = test_Y.values
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions - actuals) ** 2)
    fold_mses.append(fold_mse)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
    all_actuals.extend(actuals.tolist())

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nSingle Solvent CV MSE: {np.mean(fold_mses):.6f} ± {np.std(fold_mses):.6f}")


TASK 0: Single Solvent (Leave-One-Solvent-Out CV)


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:35<13:39, 35.65s/it]

  8%|▊         | 2/24 [01:11<13:05, 35.70s/it]

 12%|█▎        | 3/24 [01:45<12:13, 34.95s/it]

...

 92%|█████████▏| 22/24 [13:09<01:12, 36.42s/it]

 96%|█████████▌| 23/24 [13:46<00:36, 36.47s/it]

100%|██████████| 24/24 [14:22<00:00, 36.38s/it]

100%|██████████| 24/24 [14:22<00:00, 35.94s/it]


Single Solvent CV MSE: 0.009893 ± 0.009636


In [ ]:
# Run full cross-validation for full data task
print("\n" + "="*60)
print("TASK 1: Full Data (Leave-One-Ramp-Out CV)")
print("="*60)

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []
fold_mses_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='full', n_models=5, epochs=200)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    actuals = test_Y.values
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions - actuals) ** 2)
    fold_mses_full.append(fold_mse)
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
    all_actuals_full.extend(actuals.tolist())

submission_full_data = pd.DataFrame(all_predictions_full)
print(f"\nFull Data CV MSE: {np.mean(fold_mses_full):.6f} ± {np.std(fold_mses_full):.6f}")

In [ ]:
# Calculate overall score
print("\n" + "="*60)
print("OVERALL RESULTS")
print("="*60)

# Combine all predictions
all_preds_combined = submission_single_solvent[['target_1', 'target_2', 'target_3']].values.tolist() + \
                     submission_full_data[['target_1', 'target_2', 'target_3']].values.tolist()
all_actuals_combined = all_actuals + all_actuals_full

all_preds_arr = np.array(all_preds_combined)
all_actuals_arr = np.array(all_actuals_combined)

overall_mse = np.mean((all_preds_arr - all_actuals_arr) ** 2)

print(f"Single Solvent MSE: {np.mean(fold_mses):.6f}")
print(f"Full Data MSE: {np.mean(fold_mses_full):.6f}")
print(f"\nOverall MSE: {overall_mse:.6f}")

In [ ]:
# Create submission file
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

# Save submission
submission.to_csv("/home/submission/submission.csv", index=True)
print(f"Submission saved to /home/submission/submission.csv")
print(f"Submission shape: {submission.shape}")
print(submission.head())